## 🌐 **Google Drive Connection**

In [1]:
from google.colab import drive
drive.mount("/gdrive")
current_dir = "/gdrive/My\\ Drive/[2025-2026]\\ AN2DL/Challenge\\ 2"
%cd $current_dir

Mounted at /gdrive
/gdrive/My Drive/[2025-2026] AN2DL/Challenge 2


## ⚙️ **Libraries Import**

In [2]:
# Set seed for reproducibility
SEED = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
import torch
torch.manual_seed(SEED)
from torch import nn
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision.transforms import v2 as transforms
from torch.utils.data import TensorDataset, DataLoader
!pip install torchview
from torchview import draw_graph


# Configurazione di TensorBoard e directory
logs_dir = "tensorboard"
!pkill -f tensorboard
%load_ext tensorboard
!mkdir -p models

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

# Import other libraries
import cv2
import copy
import shutil
from itertools import product
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.gridspec as gridspec
import requests
from io import BytesIO
!pip install lion-pytorch
from lion_pytorch import Lion

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

PyTorch version: 2.9.0+cu126
Device: cuda


In [3]:
!pip install openai-clip
import clip
from PIL import Image
import torch

if 'device' not in globals() or not isinstance(device, torch.device):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

medical_texts = [
    "microscopic tissue",
    "histopathology slide",
    "cancer biopsy microscopic image",
]

non_medical_texts = [
    "Shrek",
    "green ogre",
    "cartoon character",
    "CGI animated ogre",
]

all_labels = medical_texts + non_medical_texts

text_tokens = clip.tokenize(all_labels).to(device)
with torch.no_grad():
    text_emb = clip_model.encode_text(text_tokens)
    text_emb /= text_emb.norm(dim=-1, keepdim=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=29b68663d56b19836e03f9504ef41d2ddc8943ce593ac745f7377b4de96c90c0
  Stored in directory: /root/.cache/pip/wheels/ab/49/bc/c2342e8e14878210ba4825cf314a53f2570f6fb18b91fce3cf
Successfully built openai-clip


100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 347MiB/s]


In [4]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Get image as numpy array (H, W, C) float32 [0,1]
        image = self.images[idx]

        # Convert numpy image to torch.Tensor (C, H, W) float32 [0,1]
        # This conversion must happen here so transforms can operate on tensors
        image = torch.from_numpy(image).permute(2, 0, 1)

        if self.transform:
            image = self.transform(image)

        if self.labels is not None:
            label = self.labels[idx]
            return image, torch.tensor(label, dtype=torch.long)
        else:
            return image

# Define augmentation transforms for training data
train_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=15), # Random rotation by +/- 15 degrees
    transforms.RandomAffine(degrees=0, translate=(0.08, 0.08), scale=(0.9, 1.1)), # Random translation up to 8% and scaling by 0.9x to 1.1x
    transforms.RandomHorizontalFlip(p=0.5), # Random horizontal flip with 50% probability
    transforms.RandomVerticalFlip(p=0.5), # Random vertical flip with 50% probability
    transforms.ToDtype(torch.float32, scale=False), # Ensure tensor is float32
])

# Define transforms for validation, test, and submission data (no augmentation)
val_test_submission_transforms = transforms.Compose([
    transforms.ToDtype(torch.float32, scale=False), # Ensure tensor is float32
])

## ⏳ **Data Loading**

In [5]:
!cp "/gdrive/My Drive/[2025-2026] AN2DL/Challenge 2/dataset/dataset.zip" /content/
!unzip -o /content/dataset.zip -d /content/dataset


# Path to your dataset folder
os.environ["DATASET_DIR"] = "/content/dataset"

TRAIN_IMAGES_DIR = os.path.join(os.environ["DATASET_DIR"], "train_data")
TEST_IMAGES_DIR  = os.path.join(os.environ["DATASET_DIR"], "test_data")
TRAIN_LABELS_PATH = os.path.join(os.environ["DATASET_DIR"], "train_labels.csv")


def load_images_from_folder(folder):
    """
    Load and preprocess images from a specified folder.

    Returns:
        np.ndarray: (N, 224, 224, 3)
    """
    images = []
    file_list = sorted(os.listdir(folder))  # keep order consistent

    for filename in file_list:
        path = os.path.join(folder, filename)

        img = cv2.imread(path)
        if img is None:
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0

        # crop center square
        h, w, _ = img.shape
        dim = min(h, w)
        img = img[(h - dim)//2:(h + dim)//2, (w - dim)//2:(w + dim)//2]

        # resize to 224x224
        img = cv2.resize(img, (224, 224))

        images.append(img)

    return np.array(images)

Archive:  /content/dataset.zip
  inflating: /content/dataset/test_data/img_0000.png  
  inflating: /content/dataset/test_data/img_0001.png  
  inflating: /content/dataset/test_data/img_0002.png  
  inflating: /content/dataset/test_data/img_0003.png  
  inflating: /content/dataset/test_data/img_0004.png  
  inflating: /content/dataset/test_data/img_0005.png  
  inflating: /content/dataset/test_data/img_0006.png  
  inflating: /content/dataset/test_data/img_0007.png  
  inflating: /content/dataset/test_data/img_0008.png  
  inflating: /content/dataset/test_data/img_0009.png  
  inflating: /content/dataset/test_data/img_0010.png  
  inflating: /content/dataset/test_data/img_0011.png  
  inflating: /content/dataset/test_data/img_0012.png  
  inflating: /content/dataset/test_data/img_0013.png  
  inflating: /content/dataset/test_data/img_0014.png  
  inflating: /content/dataset/test_data/img_0015.png  
  inflating: /content/dataset/test_data/img_0016.png  
  inflating: /content/dataset/test

In [6]:
def load_single_image(path):
    return crop_tissue_full(path)


def crop_tissue_full(path):
    # load original size
    img = cv2.imread(path)

    # convert only for processing
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)

    # mask non-gray pixels → S > 30
    mask = (S > 30).astype(np.uint8) * 255

    # smoothing the mask
    kernel = np.ones((15, 15), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        # fallback = just resize full image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        return img.astype(np.float32) / 255.0

    cnt = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)

    # crop bounding box
    crop = img[y:y+h, x:x+w]

    # resize to 224
    crop = cv2.resize(crop, (224, 224))
    crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)

    # normalize to [0,1]
    return crop.astype(np.float32) / 255.0


## 🔄 **Data Preprocessing**

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import cv2
import os
import hashlib
from collections import defaultdict

# --------------------------------------
# LOAD ALL TRAIN IMAGES
# --------------------------------------
print("Loading ALL training images...")
train_labels_df = pd.read_csv(TRAIN_LABELS_PATH)

file_list = sorted(train_labels_df["sample_index"].tolist())
print("Expected train files:", len(file_list))


# --- BAD LIST: images to remove (Shrek/mucose) ---
bad_indices = {
    5,8,22,27,36,48,62,85,95,126,129,133,136,138,148,155,159,178,179,180,187,189, #shrek
    193,196,251,254,263,286,313,319,344,346,371,376,390,393,410,415,424,443,
    459,498,499,521,540,544,547,558,565,572,586,602,607,609,614,620,623,646,658,673,
    1,12,18,20,28,44,47,52,78,90,94,130,150,161,175,184,222,268,293,333, # mucose
    342,355,368,380,407,453,463,486,497,509,530,531,533,537,557,560,567,
    578,580,603,629,635,639,643,644,645,656,657,670,675}
bad_files = {f"img_{i:04d}.png" for i in bad_indices}


# --------------------------------------
# REMOVE DUPLICATE IMAGES AUTOMATICALLY
# --------------------------------------
def md5_bytes(arr):
    return hashlib.md5(arr.tobytes()).hexdigest()

print("Searching for duplicate images...")
hash_map = defaultdict(list)

for f in file_list:
    path = os.path.join(TRAIN_IMAGES_DIR, f)
    img = cv2.imread(path)
    if img is None:
        continue
    h = md5_bytes(img)
    hash_map[h].append(f)

duplicates = [group for group in hash_map.values() if len(group) > 1]
duplicates_to_remove = set([f for group in duplicates for f in group])

print("Duplicate groups found:")
for g in duplicates:
    print(g)
print("Files to remove:", duplicates_to_remove)

file_list = [f for f in file_list if f not in duplicates_to_remove]
print("New train file count:", len(file_list))


# --------------------------------------
# Remove bad images
# --------------------------------------
file_list = [f for f in file_list if f not in bad_files]
print("After removing bad files:", len(file_list))

labels_dict = dict(zip(train_labels_df["sample_index"], train_labels_df["label"]))


# --------------------------------------
# Patches extraction function
# --------------------------------------
def extract_patches_from_image(
    img_path,
    mask_path,
    label,
    green_thr=500,
    min_tissue_area=200,   # accept only patches > 200 pixels
    return_rejected=False
):

    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    _, mask_bin = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    num_labels, labels = cv2.connectedComponents(mask_bin)

    patches = []
    labels_out = []

    rejected = []  # (patch, reason)

    for lab in range(1, num_labels):

        comp = (labels == lab).astype(np.uint8) * 255
        x, y, w, h = cv2.boundingRect(comp)

        patch = img[y:y+h, x:x+w]
        submask = comp[y:y+h, x:x+w]

        # =========================
        # 1) Tissue area
        # =========================
        tissue_area = cv2.countNonZero(submask)
        if tissue_area < min_tissue_area:
            if return_rejected:
                rejected.append((patch, "too_small"))
            continue

        # =========================
        # 2) Green mucosa filter
        # =========================
        hsv = cv2.cvtColor(patch, cv2.COLOR_RGB2HSV)
        lower = np.array([35, 40, 40])
        upper = np.array([90, 255, 255])
        mask_green = cv2.inRange(hsv, lower, upper)
        tissue_green = cv2.bitwise_and(mask_green, submask)

        if cv2.countNonZero(tissue_green) > green_thr:
            if return_rejected:
                rejected.append((patch, "green_mucosa"))
            continue

        # =========================
        # 3) Resize + save
        # =========================
        patch224 = cv2.resize(patch, (224, 224), interpolation=cv2.INTER_CUBIC)

        patches.append(patch224.astype(np.float32) / 255.0)
        labels_out.append(label)

    if return_rejected:
        return patches, labels_out, rejected
    else:
        return patches, labels_out


# --------------------------------------
# IMAGE-LEVEL TRAIN/VAL/TEST SPLIT
# --------------------------------------
unique_images = file_list
unique_labels = [labels_dict[f] for f in unique_images]


train_imgs, val_imgs = train_test_split(
    unique_images,
    test_size=0.15,  # 15% validation
    random_state=SEED,
    stratify=unique_labels
)


print("IMAGE SPLIT:")
print("Train images :", len(train_imgs))
print("Val images   :", len(val_imgs))


# --------------------------------------
# Extract patches from split
# --------------------------------------
def extract_split_patches(img_list):

    patches_all = []
    labels_all = []
    parents_all = []

    for fname in img_list:
        img_path = os.path.join(TRAIN_IMAGES_DIR, fname)
        mask_path = os.path.join(TRAIN_IMAGES_DIR, fname.replace("img", "mask"))
        label = labels_dict[fname]

        patches, patch_labels = extract_patches_from_image(img_path, mask_path, label)

        patches_all.extend(patches)
        labels_all.extend(patch_labels)

        # One parent per extracted patch
        parents_all.extend([fname] * len(patch_labels))

    return np.array(patches_all), np.array(labels_all), np.array(parents_all)


X_train, y_train, parents_train = extract_split_patches(train_imgs)
X_val,   y_val,   parents_val   = extract_split_patches(val_imgs)


print("PATCH DATASET SHAPES:")
print("Train:", X_train.shape, y_train.shape)
print("Val  :", X_val.shape,   y_val.shape)


# --------------------------------------
# ENCODING LABELS
# --------------------------------------
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_val_enc   = label_encoder.transform(y_val)

print("Class mapping:", dict(zip(label_encoder.classes_,
                                 label_encoder.transform(label_encoder.classes_))))

y_train = y_train_enc
y_val   = y_val_enc


# --------------------------------------
# LOAD EXTERNAL TEST IMAGES FOR SUBMISSION
# --------------------------------------
print("Loading external test images for submission...")
test_file_list = sorted([f for f in os.listdir(TEST_IMAGES_DIR) if f.startswith("img_")])

submission_images = np.array([
    load_single_image(os.path.join(TEST_IMAGES_DIR, f))
    for f in test_file_list
])
X_submission_images = np.array(submission_images)

print("Loaded external test images shape:", X_submission_images.shape)


# --------------------------------------
# SUMMARY
# --------------------------------------
print(f"Training Data Shape (patches): {X_train.shape}")
print(f"Training Label Shape: {y_train.shape}")
print(f"Validation Data Shape: {X_val.shape}")
print(f"Validation Label Shape: {y_val.shape}")
print(f"External Test Data Shape (images): {X_submission_images.shape}")


Loading ALL training images...
Expected train files: 691
Searching for duplicate images...
Duplicate groups found:
Files to remove: set()
New train file count: 691
After removing bad files: 581
IMAGE SPLIT:
Train images : 493
Val images   : 88
PATCH DATASET SHAPES:
Train: (3519, 224, 224, 3) (3519,)
Val  : (628, 224, 224, 3) (628,)
Class mapping: {np.str_('HER2(+)'): np.int64(0), np.str_('Luminal A'): np.int64(1), np.str_('Luminal B'): np.int64(2), np.str_('Triple negative'): np.int64(3)}
Loading external test images for submission...
Loaded external test images shape: (477, 224, 224, 3)
Training Data Shape (patches): (3519, 224, 224, 3)
Training Label Shape: (3519,)
Validation Data Shape: (628, 224, 224, 3)
Validation Label Shape: (628,)
External Test Data Shape (images): (477, 224, 224, 3)


In [8]:
# Convert numpy arrays to PyTorch datasets using the custom ImageDataset
train_ds = ImageDataset(
    X_train,
    y_train,
    transform=train_transforms
)
val_ds = ImageDataset(
    X_val,
    y_val,
    transform=val_test_submission_transforms
)

# Submission dataset for external test images (no labels)
submission_ds = ImageDataset(
    X_submission_images,
    transform=val_test_submission_transforms
)

In [9]:
# Define the input shape based on the training data
input_shape = (X_train.shape[3], X_train.shape[1], X_train.shape[2])

# Define the number of classes
num_classes = len(np.unique(y_train))

print("Input Shape:", input_shape)
print("Number of Classes:", num_classes)

Input Shape: (3, 224, 224)
Number of Classes: 4


In [10]:
# Define the batch size
BATCH_SIZE = 8

In [11]:
def make_loader(ds, batch_size, shuffle, drop_last):
    """
    Create a PyTorch DataLoader with optimized settings.

    Args:
        ds (Dataset): PyTorch Dataset object
        batch_size (int): Number of samples per batch
        shuffle (bool): Whether to shuffle data at each epoch
        drop_last (bool): Whether to drop last incomplete batch

    Returns:
        DataLoader: Configured DataLoader instance
    """
    # Determine optimal number of worker processes for data loading
    cpu_cores = os.cpu_count() or 2
    num_workers = max(2, min(4, cpu_cores))

    # Create DataLoader with performance optimizations
    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        pin_memory=True,  # Faster GPU transfer
        pin_memory_device="cuda" if torch.cuda.is_available() else "",
        prefetch_factor=4,  # Load 4 batches ahead
    )

In [12]:
# Create data loaders with different settings for each phase
train_loader = make_loader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
val_loader = make_loader(val_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
submission_loader = make_loader(submission_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

In [13]:
# Get one batch from the training data loader
for xb, yb in train_loader:
    print("Features batch shape:", xb.shape)
    print("Labels batch shape:", yb.shape)
    break  # Stop after getting one batch

Features batch shape: torch.Size([8, 3, 224, 224])
Labels batch shape: torch.Size([8])


## 🧮 **Network Parameters**

In [14]:
from sklearn.utils.class_weight import compute_class_weight

# Number of training epochs
LEARNING_RATE = 1e-4
EPOCHS = 1000
PATIENCE = 40

# Regularisation
DROPOUT_RATE = 0.2         # Dropout probability
L1_LAMBDA = 0              # L1 penalty
L2_LAMBDA = 0              # L2 penalty

# Set up loss function
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)


# Print the defined parameters
print("Epochs:", EPOCHS)
print("Batch Size:", BATCH_SIZE)
print("Learning Rate:", LEARNING_RATE)
print("Dropout Rate:", DROPOUT_RATE)
print("L1 Penalty:", L1_LAMBDA)
print("L2 Penalty:", L2_LAMBDA)

Epochs: 1000
Batch Size: 8
Learning Rate: 0.0001
Dropout Rate: 0.2
L1 Penalty: 0
L2 Penalty: 0


## 🛠️ **Build the Model**

In [15]:
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights

# ConvNeXt-Tiny model
class ConvNeXtClassifier(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.2, pretrained=True):
        super().__init__()

        self.base = convnext_tiny(
            weights=ConvNeXt_Tiny_Weights.DEFAULT if pretrained else None
        )

        # Freeze ALL layers initially (will be unfrozen progressively)
        for param in self.base.parameters():
            param.requires_grad = False

        # Replace classifier head
        in_features = self.base.classifier[2].in_features
        self.base.classifier[2] = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, num_classes)
        )

        # Always keep classifier trainable
        for param in self.base.classifier.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.base(x)

class ProgressiveUnfreezingScheduler:
    """
    Gradually unfreeze layers during training
    """
    def __init__(self, model, total_epochs):
        self.model = model
        self.total_epochs = total_epochs
        self.freeze_stages = self._get_freeze_stages(model)
        self.current_stage = -1

    def _get_freeze_stages(self, model):
        """Define which layer groups to unfreeze progressively"""
        if hasattr(model.base, 'features'):
            # ConvNeXt has 8 stages in features
            return [
                model.base.features[6:],   # Last stage (unfreeze first)
                model.base.features[4:6],  # Stage 3
                model.base.features[2:4],  # Stage 2
                model.base.features[:2],   # Stage 1 (unfreeze last)
            ]
        return []

    def step(self, epoch):
        """Unfreeze one stage every 20 epochs"""
        if len(self.freeze_stages) == 0:
            return

        # Aggressive schedule for early stopping scenarios
        unfreeze_schedule = [0, 20, 40, 60]  # Epochs to unfreeze each stage

        for i, trigger_epoch in enumerate(unfreeze_schedule):
            if epoch == trigger_epoch and i > self.current_stage:
                self.current_stage = i

                # Unfreeze this stage
                for param in self.freeze_stages[i].parameters():
                    param.requires_grad = True

                print(f"  🔓 Unfroze stage {i + 1}/{len(self.freeze_stages)}")
                break

def get_layer_groups_convnext(model):
    """
    Split ConvNeXt into layer groups for discriminative learning rates
    """
    if hasattr(model.base, 'features'):
        return [
            model.base.features[:2],   # Early stages (stem + stage1)
            model.base.features[2:4],  # Stage 2
            model.base.features[4:6],  # Stage 3
            model.base.features[6:],   # Stage 4 (late stages)
            model.base.classifier,     # Head
        ]
    return []

def get_optimizer_with_discriminative_lr(model, base_lr=1e-4, weight_decay=0):
    """
    Lower learning rates for early layers, higher for later layers
    """
    layer_groups = get_layer_groups_convnext(model)

    # Learning rate multipliers: [0.1x, 0.3x, 0.5x, 0.8x, 1x]
    lr_multipliers = [0.1, 0.3, 0.5, 0.8, 1.0]

    param_groups = []
    for i, (group, mult) in enumerate(zip(layer_groups, lr_multipliers)):
        params = [p for p in group.parameters() if p.requires_grad]
        if len(params) > 0:
            param_groups.append({
                'params': params,
                'lr': base_lr * mult,
                'weight_decay': weight_decay
            })
            print(f"  Layer group {i+1}: LR = {base_lr * mult:.2e}, {len(params)} params")

    return Lion(param_groups, lr=base_lr)

print(f"✅ ConvNeXt model class defined with {num_classes} classes")

✅ ConvNeXt model class defined with 4 classes


## 🧠 **Model Training**

In [16]:
# Enable mixed precision training for GPU acceleration (reused across folds)
scaler = torch.amp.GradScaler(enabled=(device.type == 'cuda'))

print("✅ Training setup ready")

✅ Training setup ready


In [17]:
def train_one_epoch(model, train_loader, criterion, optimizer, scaler, device, l1_lambda=0, l2_lambda=0):
    """
    Perform one complete training epoch through the entire training dataset.

    Args:
        model (nn.Module): The neural network model to train
        train_loader (DataLoader): PyTorch DataLoader containing training data batches
        criterion (nn.Module): Loss function (e.g., CrossEntropyLoss, MSELoss)
        optimizer (torch.optim): Optimization algorithm (e.g., Adam, SGD)
        scaler (GradScaler): PyTorch's gradient scaler for mixed precision training
        device (torch.device): Computing device ('cuda' for GPU, 'cpu' for CPU)
        l1_lambda (float): Lambda for L1 regularization
        l2_lambda (float): Lambda for L2 regularization

    Returns:
        tuple: (average_loss, f1 score) - Training loss and f1 score for this epoch
    """
    model.train()  # Set model to training mode

    running_loss = 0.0
    all_predictions = []
    all_targets = []

    # Iterate through training batches
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # Move data to device (GPU/CPU)
        inputs, targets = inputs.to(device), targets.to(device)

        # Clear gradients from previous step
        optimizer.zero_grad(set_to_none=True)

        # Forward pass with mixed precision (if CUDA available)
        with torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
            logits = model(inputs)
            loss = criterion(logits, targets)

            # Add L1 and L2 regularization
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            l2_norm = sum(p.pow(2).sum() for p in model.parameters())
            loss = loss + l1_lambda * l1_norm + l2_lambda * l2_norm

        # Backward pass with gradient scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Accumulate metrics
        running_loss += loss.item() * inputs.size(0)
        predictions = logits.argmax(dim=1)
        all_predictions.append(predictions.cpu().numpy())
        all_targets.append(targets.cpu().numpy())

    # Calculate epoch metrics
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_f1 = f1_score(
        np.concatenate(all_targets),
        np.concatenate(all_predictions),
        average='weighted'
    )

    return epoch_loss, epoch_f1

In [18]:
def validate_one_epoch(model, val_loader, criterion, device):
    """
    Perform one complete validation epoch through the entire validation dataset.

    Args:
        model (nn.Module): The neural network model to evaluate (must be in eval mode)
        val_loader (DataLoader): PyTorch DataLoader containing validation data batches
        criterion (nn.Module): Loss function used to calculate validation loss
        device (torch.device): Computing device ('cuda' for GPU, 'cpu' for CPU)

    Returns:
        tuple: (average_loss, accuracy) - Validation loss and accuracy for this epoch

    Note:
        This function automatically sets the model to evaluation mode and disables
        gradient computation for efficiency during validation.
    """
    model.eval()  # Set model to evaluation mode

    running_loss = 0.0
    all_predictions = []
    all_targets = []

    # Disable gradient computation for validation
    with torch.no_grad():
        for inputs, targets in val_loader:
            # Move data to device
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass with mixed precision (if CUDA available)
            with torch.amp.autocast(device_type=device.type, enabled=(device.type == 'cuda')):
                logits = model(inputs)
                loss = criterion(logits, targets)

            # Accumulate metrics
            running_loss += loss.item() * inputs.size(0)
            predictions = logits.argmax(dim=1)
            all_predictions.append(predictions.cpu().numpy())
            all_targets.append(targets.cpu().numpy())

    # Calculate epoch metrics
    epoch_loss = running_loss / len(val_loader.dataset)
    epoch_accuracy = f1_score(
        np.concatenate(all_targets),
        np.concatenate(all_predictions),
        average='weighted'
    )

    return epoch_loss, epoch_accuracy

In [19]:
def log_metrics_to_tensorboard(writer, epoch, train_loss, train_f1, val_loss, val_f1, model):
    """
    Log training metrics and model parameters to TensorBoard for visualization.

    Args:
        writer (SummaryWriter): TensorBoard SummaryWriter object for logging
        epoch (int): Current epoch number (used as x-axis in TensorBoard plots)
        train_loss (float): Training loss for this epoch
        train_f1 (float): Training f1 score for this epoch
        val_loss (float): Validation loss for this epoch
        val_f1 (float): Validation f1 score for this epoch
        model (nn.Module): The neural network model (for logging weights/gradients)

    Note:
        This function logs scalar metrics (loss/f1 score) and histograms of model
        parameters and gradients, which helps monitor training progress and detect
        issues like vanishing/exploding gradients.
    """
    # Log scalar metrics
    writer.add_scalar('Loss/Training', train_loss, epoch)
    writer.add_scalar('Loss/Validation', val_loss, epoch)
    writer.add_scalar('F1/Training', train_f1, epoch)
    writer.add_scalar('F1/Validation', val_f1, epoch)

    # Log model parameters and gradients
    for name, param in model.named_parameters():
        if param.requires_grad:
            # Check if the tensor is not empty before adding a histogram
            if param.numel() > 0:
                writer.add_histogram(f'{name}/weights', param.data, epoch)
            if param.grad is not None:
                # Check if the gradient tensor is not empty before adding a histogram
                if param.grad.numel() > 0:
                    if param.grad is not None and torch.isfinite(param.grad).all():
                        writer.add_histogram(f'{name}/gradients', param.grad.data, epoch)

In [20]:
def fit(model, train_loader, val_loader, epochs, criterion, optimizer, scaler, device,
        l1_lambda=0, l2_lambda=0, patience=0, evaluation_metric="val_f1", mode='max',
        restore_best_weights=True, writer=None, verbose=10, experiment_name="",
        unfreeze_scheduler=None):
    """
    Train the neural network model on the training data and validate on the validation data.

    Args:
        model (nn.Module): The neural network model to train
        train_loader (DataLoader): PyTorch DataLoader containing training data batches
        val_loader (DataLoader): PyTorch DataLoader containing validation data batches
        epochs (int): Number of training epochs
        criterion (nn.Module): Loss function (e.g., CrossEntropyLoss, MSELoss)
        optimizer (torch.optim): Optimization algorithm (e.g., Adam, SGD)
        scaler (GradScaler): PyTorch's gradient scaler for mixed precision training
        device (torch.device): Computing device ('cuda' for GPU, 'cpu' for CPU)
        l1_lambda (float): L1 regularization coefficient (default: 0)
        l2_lambda (float): L2 regularization coefficient (default: 0)
        patience (int): Number of epochs to wait for improvement before early stopping (default: 0)
        evaluation_metric (str): Metric to monitor for early stopping (default: "val_f1")
        mode (str): 'max' for maximizing the metric, 'min' for minimizing (default: 'max')
        restore_best_weights (bool): Whether to restore model weights from best epoch (default: True)
        writer (SummaryWriter, optional): TensorBoard SummaryWriter object for logging (default: None)
        verbose (int, optional): Frequency of printing training progress (default: 10)
        experiment_name (str, optional): Experiment name for saving models (default: "")

    Returns:
        tuple: (model, training_history) - Trained model and metrics history
    """

    # Initialize metrics tracking
    training_history = {
        'train_loss': [], 'val_loss': [],
        'train_f1': [], 'val_f1': []
    }

    # Configure early stopping if patience is set
    if patience > 0:
        patience_counter = 0
        best_metric = float('-inf') if mode == 'max' else float('inf')
        best_epoch = 0

    print(f"Training {epochs} epochs...")

    # Main training loop: iterate through epochs
    for epoch in range(1, epochs + 1):

        # Progressive unfreezing
        if unfreeze_scheduler is not None:
            unfreeze_scheduler.step(epoch - 1)  # epoch-1 because we start from 1

        # Forward pass through training data, compute gradients, update weights
        train_loss, train_f1 = train_one_epoch(
            model, train_loader, criterion, optimizer, scaler, device, l1_lambda, l2_lambda
        )

        # Evaluate model on validation data without updating weights
        val_loss, val_f1 = validate_one_epoch(
            model, val_loader, criterion, device
        )

        # Store metrics for plotting and analysis
        training_history['train_loss'].append(train_loss)
        training_history['val_loss'].append(val_loss)
        training_history['train_f1'].append(train_f1)
        training_history['val_f1'].append(val_f1)

        # Write metrics to TensorBoard for visualization
        if writer is not None:
            log_metrics_to_tensorboard(
                writer, epoch, train_loss, train_f1, val_loss, val_f1, model
            )

        # Print progress every N epochs or on first epoch
        if verbose > 0:
            if epoch % verbose == 0 or epoch == 1:
                print(f"Epoch {epoch:3d}/{epochs} | "
                    f"Train: Loss={train_loss:.4f}, F1 Score={train_f1:.4f} | "
                    f"Val: Loss={val_loss:.4f}, F1 Score={val_f1:.4f}")

        # Early stopping logic: monitor metric and save best model
        if patience > 0:
            current_metric = training_history[evaluation_metric][-1]
            is_improvement = (current_metric > best_metric) if mode == 'max' else (current_metric < best_metric)

            if is_improvement:
                best_metric = current_metric
                best_epoch = epoch
                torch.save(model.state_dict(), "models/"+experiment_name+'_model.pt')
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping triggered after {epoch} epochs.")
                    break

    # Restore best model weights if early stopping was used
    if restore_best_weights and patience > 0:
        model.load_state_dict(torch.load("models/"+experiment_name+'_model.pt'))
        print(f"Best model restored from epoch {best_epoch} with {evaluation_metric} {best_metric:.4f}")

    # Save final model if no early stopping
    if patience == 0:
        torch.save(model.state_dict(), "models/"+experiment_name+'_model.pt')

    # Close TensorBoard writer
    if writer is not None:
        writer.close()

    return model, training_history

In [21]:
%%time

# Create model
fold_model = ConvNeXtClassifier(
    num_classes=num_classes,
    dropout_rate=DROPOUT_RATE,
    pretrained=True
).to(device)

print(f"\n📊 Model architecture: ConvNeXt-Tiny")

# Create optimizer with discriminative learning rates
print(f"\n🎯 Setting up discriminative learning rates:")
optimizer = get_optimizer_with_discriminative_lr(
    fold_model,
    base_lr=LEARNING_RATE,
    weight_decay=L2_LAMBDA
)

# Create progressive unfreezing scheduler
unfreeze_scheduler = ProgressiveUnfreezingScheduler(fold_model, EPOCHS)

# Set up TensorBoard
experiment_name = "convnext_tiny_with_test"
writer = SummaryWriter(f"./{logs_dir}/{experiment_name}")

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Train model
print(f"\n🚀 Starting training with progressive unfreezing...")
fold_model, fold_history = fit(
    model=fold_model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=EPOCHS,
    criterion=criterion,
    optimizer=optimizer,
    scaler=scaler,
    device=device,
    writer=writer,
    verbose=1,
    experiment_name=experiment_name,
    patience=PATIENCE,
    unfreeze_scheduler=unfreeze_scheduler
)

# Calculate final F1
final_val_f1 = round(max(fold_history['val_f1']) * 100, 2)
print(f'\n✅ Final validation F1 score: {final_val_f1}%')

# Save for later evaluation
vanilla_model = fold_model
vanilla_history = fold_history

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth


100%|██████████| 109M/109M [00:00<00:00, 245MB/s] 



📊 Model architecture: ConvNeXt-Tiny

🎯 Setting up discriminative learning rates:
  Layer group 5: LR = 1.00e-04, 4 params

🚀 Starting training with progressive unfreezing...
Training 1000 epochs...
  🔓 Unfroze stage 1/4
Epoch   1/1000 | Train: Loss=1.3763, F1 Score=0.3154 | Val: Loss=1.3162, F1 Score=0.3553
Epoch   2/1000 | Train: Loss=1.3201, F1 Score=0.3700 | Val: Loss=1.3006, F1 Score=0.4078
Epoch   3/1000 | Train: Loss=1.3032, F1 Score=0.3936 | Val: Loss=1.2876, F1 Score=0.3709
Epoch   4/1000 | Train: Loss=1.2891, F1 Score=0.4014 | Val: Loss=1.3221, F1 Score=0.3294
Epoch   5/1000 | Train: Loss=1.2770, F1 Score=0.3980 | Val: Loss=1.3650, F1 Score=0.3280
Epoch   6/1000 | Train: Loss=1.2891, F1 Score=0.4010 | Val: Loss=1.3188, F1 Score=0.3324
Epoch   7/1000 | Train: Loss=1.2784, F1 Score=0.4087 | Val: Loss=1.2796, F1 Score=0.3833
Epoch   8/1000 | Train: Loss=1.2817, F1 Score=0.4016 | Val: Loss=1.2699, F1 Score=0.3878
Epoch   9/1000 | Train: Loss=1.2780, F1 Score=0.4109 | Val: Loss=1.

In [22]:
import torch
import numpy as np
import pandas as pd
import cv2
import os
import torch.nn.functional as F

# ============================================================
# 1) EXTRACT PATCHES FROM SUBMISSION IMAGES
# ============================================================

def extract_submission_patches(
    green_thr=500,
    min_tissue_area=200
):

    submission_patches = []
    parents_submission = []

    for fname in test_file_list:

        img_path = os.path.join(TEST_IMAGES_DIR, fname)
        mask_path = os.path.join(TEST_IMAGES_DIR, fname.replace("img", "mask"))

        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        _, mask_bin = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        num_labels, labels = cv2.connectedComponents(mask_bin)

        accepted_any = False
        best_fallback = None
        best_area = 0

        for lab in range(1, num_labels):

            comp = (labels == lab).astype(np.uint8) * 255
            x, y, w, h = cv2.boundingRect(comp)

            patch = img[y:y+h, x:x+w]
            submask = comp[y:y+h, x:x+w]

            tissue_area = cv2.countNonZero(submask)

            # ======================
            # FALLBACK TRACKING
            # ======================
            if tissue_area > best_area:
                best_area = tissue_area
                best_fallback = patch

            # ======================
            # AREA FILTER
            # ======================
            if tissue_area < min_tissue_area:
                continue

            # ======================
            # ACCEPT PATCH
            # ======================
            patch224 = cv2.resize(patch, (224, 224), interpolation=cv2.INTER_CUBIC)
            submission_patches.append(patch224.astype(np.float32) / 255.0)
            parents_submission.append(fname)

            accepted_any = True

        # ============================
        # FALLBACK IF NONE ACCEPTED
        # ============================
        if not accepted_any and best_fallback is not None:
            patch224 = cv2.resize(best_fallback, (224, 224), interpolation=cv2.INTER_CUBIC)
            submission_patches.append(patch224.astype(np.float32) / 255.0)
            parents_submission.append(fname)

    return np.array(submission_patches), np.array(parents_submission)


X_sub, parents_sub = extract_submission_patches()

print("SUBMISSION PATCHES:", X_sub.shape)
print("Parents:", len(parents_sub))


# ============================================================
# 2) MODEL PREDICTION — COLLECT LOGITS
# ============================================================

vanilla_model.eval()
batch_size = 32

all_logits = []

with torch.no_grad():
    for i in range(0, len(X_sub), batch_size):
        batch = torch.tensor(X_sub[i:i+batch_size]).permute(0, 3, 1, 2).to(device)

        logits = vanilla_model(batch)       # (B, num_classes)
        all_logits.append(logits.cpu().numpy())

all_logits = np.vstack(all_logits)         # (num_patches, num_classes)

# Softmax probabilities
probs = F.softmax(torch.tensor(all_logits), dim=1).numpy()

num_classes = probs.shape[1]
prob_cols = [f"p{i}" for i in range(num_classes)]


# ============================================================
# 3) BUILD DATAFRAME
# ============================================================

df_sub = pd.DataFrame(
    np.column_stack([parents_sub, probs]),
    columns=["filename"] + prob_cols
)

df_sub[prob_cols] = df_sub[prob_cols].astype(float)


# ============================================================
# 4) SOFTMAX WEIGHTED PER IMAGE
# ============================================================

image_level_probs = df_sub.groupby("filename")[prob_cols].mean()

image_level_pred = image_level_probs.values.argmax(axis=1)
image_level_pred = pd.Series(image_level_pred, index=image_level_probs.index)

# Convert to string labels
final_labels = label_encoder.inverse_transform(image_level_pred.values)


# ============================================================
# 5) SAVE SUBMISSION
# ============================================================

submission_df = pd.DataFrame({
    "sample_index": image_level_pred.index,
    "label": final_labels
})

submission_df.to_csv("submissionProva4.csv", index=False)

print("\n🎯 DONE — submission.csv created with SOFTMAX WEIGHTING!")
print(submission_df.head())
print("Total images:", len(submission_df))


SUBMISSION PATCHES: (3337, 224, 224, 3)
Parents: 3337

🎯 DONE — submission.csv created with SOFTMAX WEIGHTING!
   sample_index      label
0  img_0000.png  Luminal B
1  img_0001.png  Luminal B
2  img_0002.png  Luminal B
3  img_0003.png  Luminal A
4  img_0004.png  Luminal B
Total images: 477
